In [1]:
import pandas as pd
import numpy as np

In [2]:
credit=pd.read_csv("tmdb_5000_credits.csv")
credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
movies=pd.read_csv("tmdb_5000_movies.csv")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
# we need to merge the dataset before the use 
# movies.shape
movies=movies.merge(credit, on="title")
movies.shape

(4809, 23)

In [5]:
movies.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [6]:
# important colums for tag formation 
# genres ,id ,keyword,title,overview ,production_companies,cast (top 3 cast of movie) ,crew(director)
movies["genres"].values

array(['[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]',
       '[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]',
       '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]',
       ...,
       '[{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}, {"id": 10770, "name": "TV Movie"}]',
       '[]', '[{"id": 99, "name": "Documentary"}]'], dtype=object)

In [7]:
# kepping the required columns 
movies=movies[["movie_id","title","genres","keywords","overview","cast","crew"]]
movies.head(1)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
movies.isnull().sum()
movies.dropna(inplace=True) #deleting the rows with NULL VALUES 
movies.isnull().sum()

movie_id    0
title       0
genres      0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

WE NEED DATAFRAME AS 
movie_id  title  tag(genres	keywords	overview	cast	crew)


In [9]:
# firstly we need to convert the dictionary to list formate (genres,keyword ,overview,production )
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# ->convert TO 
# ["Action",Adventure",Fantasy","Science Fiction"]
import ast


def convert(obj): 
    l=[]
    obj=ast.literal_eval(obj)  #To convert any string literal to any object type (LIST)
    for A in obj:
        l.append(A["name"])
    return l
        

In [10]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [11]:
movies["genres"]=movies["genres"].apply(convert)
movies["keywords"]=movies["keywords"].apply(convert)



In [12]:
# TO make cast of First 3 cast member 
def convert_3(obj): 
    l=[]
    count=0
    obj=ast.literal_eval(obj)  #To convert any string literal to any object type (LIST)
    for A in obj:
        if count<3:
            l.append(A["name"])
            count+=1
        else:
            break
    return l
        

In [13]:
movies["cast"]=movies["cast"].apply(convert_3)

In [14]:
movies["crew"][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [15]:
# we need to have the director name as crew column
def getCrew(obj): 
    l=[]
    obj=ast.literal_eval(obj)  #To convert any string literal to any object type (LIST)
    for A in obj:
        if(A["job"]=="Director"):
            l.append(A["name"])
        
    return l
        

In [16]:
movies["crew"]=movies["crew"].apply(getCrew)

In [17]:
# overview ko bhi convert to list
movies["overview"]=movies["overview"].apply(lambda x:x.split())

In [18]:
movies.head(1)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [19]:
# tag formation
movies["tag"]=movies["genres"]+movies["keywords"]+movies["overview"]+movies["cast"]+movies["crew"]

In [20]:
movies=movies[["movie_id","title","tag"]]

In [21]:
movies["tag"]=movies["tag"].apply(lambda x:[i.replace(" ","") for i in x])  #To remove the spaces inorder to have an better recommendation system 

In [22]:
movies

,movie_id,title,tag
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."
...,...,...,...
4804,9367,El Mariachi,"[Action, Crime, Thriller, unitedstates–mexicob..."
4805,72766,Newlyweds,"[Comedy, Romance, A, newlywed, couple's, honey..."
4806,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TVMovie, date, loveat..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [23]:
movies["tag"] =movies["tag"].apply(lambda x:" ".join(x))

In [24]:
new_df=movies
new_df

,movie_id,title,tag
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...
...,...,...,...
4804,9367,El Mariachi,Action Crime Thriller unitedstates–mexicobarri...
4805,72766,Newlyweds,Comedy Romance A newlywed couple's honeymoon i...
4806,231617,"Signed, Sealed, Delivered",Comedy Drama Romance TVMovie date loveatfirsts...
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [25]:
# lower the tags ==
new_df["tag"]=new_df["tag"].apply(lambda x:x.lower())
new_df.sample(10)

,movie_id,title,tag
4402,91122,16 to Life,comedy romance affection womandirector romanti...
183,101299,The Hunger Games: Catching Fire,adventure action sciencefiction competition ba...
1127,136795,The Heat,action comedy crime boston fbi buddycomedy upt...
1880,5123,August Rush,drama date lossofson loveatfirstsight childlab...
865,11321,Seven Pounds,drama vegetarian taxcollector pianist blind or...
4037,17334,The Other End of the Line,comedy romance indianlead callcenter romanticc...
2422,38117,Beastly,drama fantasy romance basedonnovel love beauti...
3736,26379,Paa,drama family foreign he suffers from a progeri...
161,205584,Gods of Egypt,fantasy egypt underworld fight mythology nile ...
3918,29463,Krush Groove,"drama music romance russell walker is a young,..."


In [41]:
# vectorization 
# using the sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english")
X=cv.fit_transform(new_df["tag"])
# cv.get_feature_names_out(X)

vector=cv.fit_transform(new_df["tag"]).toarray()

In [42]:
vector[0] # vector for movie 1 

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [43]:
cv.get_feature_names_out()[np.arange(20,100)]
# 'accident', 'accidental', 'accidentally'---> accident hona chahiye but differrnt hai to hu,ko isko convert karna hoga 
# applying steming 
# using library nltk

array(['1944', '1950', '1950s', '1960', '1960s', '1970', '1970s', '1971',
       '1974', '1976', '1980', '1985', '1990', '1999', '19th',
       '19thcenturi', '20', '200', '2003', '2009', '20th', '21st', '23',
       '24', '25', '30', '300', '3d', '40', '50', '500', '60', '70', '80',
       'aaron', 'aaroneckhart', 'abandon', 'abduct', 'abigailbreslin',
       'abil', 'abl', 'aboard', 'abov', 'abus', 'academ', 'academi',
       'accept', 'access', 'accid', 'accident', 'acclaim', 'accompani',
       'accomplish', 'account', 'accus', 'ace', 'achiev', 'acquaint',
       'act', 'action', 'actionhero', 'activ', 'activist', 'activities',
       'actor', 'actress', 'actual', 'ad', 'adam', 'adamsandl',
       'adamshankman', 'adapt', 'add', 'addict', 'adjust', 'admir',
       'admit', 'adolesc', 'adopt', 'ador'], dtype=object)

In [29]:
# pip install nltk

In [30]:

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
ps.stem('accidentally')

'accident'

In [31]:
def stem(text):
    y=[]
    for A in text.split():
        y.append(ps.stem(A))
    return " ".join(y)


    

In [32]:
new_df["tag"][0]

'action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. samworthington zoesaldana sigourneyweaver jamescameron'

In [33]:
stem(new_df["tag"][0])  #ediited version

'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. samworthington zoesaldana sigourneyweav jamescameron'

In [34]:
new_df["tag"]=new_df["tag"].apply(lambda x:stem(x))

In [39]:
new_df["tag"][0] # Now no problem of multiple word in vectorization

'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. samworthington zoesaldana sigourneyweav jamescameron'

In [36]:
# we need to find the simalarities bewteen every movies 
#  therefore we will use the cosine values as similarities 

from sklearn.metrics.pairwise import cosine_similarity

In [44]:
similarity=cosine_similarity(vector)

In [45]:
similarity.shape

(4806, 4806)

In [46]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [47]:
list(enumerate(similarity[0]))   # index wise list 

[(0, 1.0000000000000002),
 (1, 0.08346223261119858),
 (2, 0.08603090020146065),
 (3, 0.0734718358370645),
 (4, 0.1892994097121204),
 (5, 0.10838874619051501),
 (6, 0.04024218182927669),
 (7, 0.14673479641335554),
 (8, 0.05923488777590923),
 (9, 0.0967301666813349),
 (10, 0.10259783520851541),
 (11, 0.09464970485606021),
 (12, 0.09037128496931669),
 (13, 0.04499212706658476),
 (14, 0.12824729401064427),
 (15, 0.06282808624375433),
 (16, 0.07894736842105264),
 (17, 0.13977653617040256),
 (18, 0.09493290614465533),
 (19, 0.0830812984794528),
 (20, 0.058038100008800934),
 (21, 0.10968169942141635),
 (22, 0.0662266178532522),
 (23, 0.08740748201220976),
 (24, 0.0533380747062665),
 (25, 0.05101627678885769),
 (26, 0.15389675281277312),
 (27, 0.18693292157876878),
 (28, 0.116543309349613),
 (29, 0.065033247714309),
 (30, 0.06684847767323797),
 (31, 0.15907119074394446),
 (32, 0.08520286456846099),
 (33, 0.09733285267845754),
 (34, 0.0),
 (35, 0.09933992677987831),
 (36, 0.17119059581558146),


In [48]:
#create a function 
def recomendation(movie):
    movie_index=new_df[new_df["title"]==movie].index[0]  # returns movies index in dataframe 
    distance=similarity[movie_index]
    movie_list=sorted(list(enumerate(similarity[0])) ,reverse=True,key=lambda x:x[1])[1:6]
    for A in movie_list:
        print(new_df.iloc[A[0]].title)
    
    

In [49]:
recomendation("Avatar")

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [50]:
#Now we need to have website 

similarity


array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [51]:
import pickle as pk
pk.dump(new_df.to_dict(),open("movies.pkl","wb"))
# pk.dump(similarity,open("similarity.pkl.exe","wb"))

In [52]:
new_df

,movie_id,title,tag
0,19995,Avatar,action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drugabus exotici...
2,206647,Spectre,action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises,action crime drama thriller dccomic crimefight...
4,49529,John Carter,action adventur sciencefict basedonnovel mar m...
...,...,...,...
4804,9367,El Mariachi,action crime thriller unitedstates–mexicobarri...
4805,72766,Newlyweds,comedi romanc a newlyw couple' honeymoon is up...
4806,231617,"Signed, Sealed, Delivered",comedi drama romanc tvmovi date loveatfirstsig...
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [54]:
pk.dump(similarity,open("similarity.pkl","wb"))